## Классификация отзыва положительный / отрицательный по его текстовому содержания

dataset: https://www.kaggle.com/iarunava/imdb-movie-reviews-dataset/

### подготовка датасета

загрузка тренировочного датасета:

In [116]:
from sklearn.datasets import load_files
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import random
from sklearn.ensemble import RandomForestClassifier

In [117]:
reviews_train = load_files('input/aclimdb/train/')

In [118]:
text_train, y_train = reviews_train.data, reviews_train.target

из датасета можно выделить две сущности <s>в виде гномика</s> - сам текст отзыва и его оценка:

In [119]:
text_train[0]

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."

оценка бинарная - негативный / положительный отзыв

In [120]:
np.unique (y_train)

array([0, 1])

проверка количества документов и количества выборок в каждом классе train dataset:

In [121]:
print(f'кол-во документов в выборке: {len(text_train)}')

кол-во документов в выборке: 25000


In [122]:
print(f'кол-во выборок в каждом классе: {np.bincount(y_train)}') 

кол-во выборок в каждом классе: [12500 12500]


аналогичная обработка test dataset:

In [123]:
reviews_test = load_files('input/aclimdb/test/')

In [124]:
text_test, y_test = reviews_test.data, reviews_test.target
print(f'кол-во документов в выборке: {len(text_test)}')
print(f'кол-во выборок в каждом классе: {np.bincount(y_test)}') 

кол-во документов в выборке: 25000
кол-во выборок в каждом классе: [12500 12500]


Перевод текста в разряженную матрицу типа bag_of_words, при этом зададим параметры:

min_df - минимальная частота повторения слова для определения как признака; слова, которые имеют частоту повторения в документе ниже, чем указанный порог, будут опускаться 

ngram_range - длина токена

In [125]:
# инициализация конвертора
vect = CountVectorizer(min_df=5, ngram_range=(2, 2))

In [126]:
# конвертация текста в матрицу
X_train = vect.fit(text_train).transform(text_train)
X_test = vect.transform(text_test)

In [127]:
print("размер полученного словаря: {}".format(len(vect.vocabulary_)))
print("описание X_trian:\n{}".format(repr(X_train)))
print("описание X_test:\n{}".format(repr(X_test)))

размер полученного словаря: 129549
описание X_trian:
<25000x129549 sparse matrix of type '<class 'numpy.int64'>'
	with 3607330 stored elements in Compressed Sparse Row format>
описание X_test:
<25000x129549 sparse matrix of type '<class 'numpy.int64'>'
	with 3392376 stored elements in Compressed Sparse Row format>


в результате конвертации получаем перечень признаков, по которым будет проводиться обучение:

In [128]:
feature_names = vect.get_feature_names()
print('кол-во признаков: {}'.format(len(feature_names)))

кол-во признаков: 129549


### построение модели

тюнинг модели через поиск по сетке

In [129]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

print("лучший результат на кросс-валидации: {:.2f}".format(grid.best_score_))
print("наиболее оптимальные параметры: ", grid.best_params_)
print("наиболее оптимальный коэффициент модели: ", grid.best_estimator_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


лучший результат на кросс-валидации: 0.88
наиболее оптимальные параметры:  {'C': 1}
наиболее оптимальный коэффициент модели:  LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


### обучение и прогнозирование

с учетом наиболее оптимальных параметров обучаем модели и запускамем прогноз для тестовой выборки:

In [130]:
lr = grid.best_estimator_
lr.fit(X_train, y_train)
lr.predict(X_test)
print('точность предсказания: {:.2f}'.format(lr.score(X_test, y_test)))

точность предсказания: 0.88


In [131]:
lr_predictions = lr.predict(X_test)

в качестве примера отзыва - предсказания:

In [132]:
tst = [random.randint(1, len(lr_predictions)) for i in range(4)]
for i in tst:
    print(lr_predictions[i],'\n', text_test[i],'\n')

0 
 b'I\'ve seen some terrible book-to-film adaptations in my day, but this one tops them all! The bizarrely unattractive cast detracts from the story, which is, in itself, untrue to the book. Mr. Tilney is nothing like handsome; as for Catherine Morland, a rat-like appearance makes this heroine a difficult one to sell to a sympathetic audience. Isabella is nothing like the Aphrodite one reads about in the original text, and James Morland appears in the film far too little to leave the viewer with any understanding of his important role in the story. Also, as others have pointed out before, this novel was intended to satirize the Gothic craze prevalent in Austen\'s time, but it appears that this "soft horror" film was designed and meant to be taken seriously. I\'m sure Jane Austen turns over in her grave each time one of her fans is disappointed by this awful interpretation of what was supposed to be a joke.' 

0 
 b'Seriously, I don\xc2\xb4t really get why people here are bashing it. 

с первого взгляда оценки соответствуют классификации

### применение xgboost

сравнение лог регрессии и классификатором xgboost

инициализирую, обучаю модель и запускаю прогноз

In [133]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
print(model)
xgb_predictions = xgb_model.predict(X_test)
print('точность предсказания: {:.2f}'.format(xgb_model.score(X_test, y_test)))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
точность предсказания: 0.76


In [134]:
print('точность предсказания: {:.2f}'.format(xgb_model.score(X_test, y_test)))

точность предсказания: 0.76


странно, но лог регрессия показала себя лучше xgboost

### применение RandomForestClassifier

сравнение лог регрессии и RandomForestClassifier

In [135]:
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, y_train)
print(model)
rfc_predictions = rfc_model.predict(X_test)
print('точность предсказания: {:.2f}'.format(rfc_model.score(X_test, y_test)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
точность предсказания: 0.75


точность также оказалась ниже точности по логрегресии

определенно, по направлению прогнозирования класса текстовых отзывов можно было почистить признаки, и посмотреть, что получится. поэтому как следующей шаг я возьму эту задачу.